In [27]:
import os
import openai
import json
import glob

In [10]:
OPENAI_API_KEY="sk-"

In [11]:
#openai.organization = "org-cjJTcTfsM9bxpOO9ia56FNLw"
openai.api_key = OPENAI_API_KEY
openai.Model.list()

<OpenAIObject list at 0x10af3d180> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 16690855

## Generate training data

In [47]:
def generate_train_data(prompt: str, file: str):
    print(file)
    with open(file, "r") as f:
        return { 
            "prompt": prompt,
            "completion": f" {f.read()}" # The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
        }


training_data = []

files = glob.glob('../seeve/*/src/*.wat', 
                   recursive = True)
i = 0
for file in files:
    i = i + 1
    training_data.append(generate_train_data(f"bad code {i}\n\n###\n\n", file))
  
training_data

with open("training_data.jsonl", "w") as f:
    for e in training_data:
        json.dump(e, f)
        f.write("\n")


../seeve/CWE-457/src/test1.wat
../seeve/CWE-665/src/test1.wat
../seeve/CWE-125/src/test4.wat
../seeve/CWE-125/src/test1.wat
../seeve/CWE-125/src/test3.wat
../seeve/CWE-125/src/test2.wat
../seeve/CWE-170/src/test1.wat
../seeve/CWE-190/src/test1.wat
../seeve/CWE-369/src/test4.wat
../seeve/CWE-369/src/test1.wat
../seeve/CWE-369/src/test3.wat
../seeve/CWE-369/src/test2.wat
../seeve/CWE-197/src/test1.wat
../seeve/CWE-119/src/test4.wat
../seeve/CWE-119/src/test1.wat
../seeve/CWE-119/src/test3.wat
../seeve/CWE-119/src/test2.wat
../seeve/CWE-120/src/test1.wat
../seeve/CWE-134/src/test1.wat
../seeve/CWE-134/src/test2.wat
../seeve/CWE-193/src/test1.wat
../seeve/CWE-20/src/test1.wat
../seeve/CWE-20/src/test2.wat
../seeve/CWE-195/src/test1.wat
../seeve/CWE-415/src/test1.wat
../seeve/CWE-476/src/test1.wat


In [50]:
!openai tools fine_tunes.prepare_data -f "training_data.jsonl"


/Users/sjust228/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Analyzing...

- Your file contains 26 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- There are 12 examples that are very long. These are rows: [1, 7, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- All prompts end with suffix `\n\n###\n\n`
- All prompts start with prefix `bad code `
- All completions start with prefix ` (module
  (type (;0;) (func (param i32`. Most of the time you should only add the output data into the completion, without any prefix
- All completions end wit